# Create Pivot Tables

Create a pivot table of patients and conditions. Then combine all individuals and their variants into a single table for analysis.

In [12]:
import pandas as pd
import sqlalchemy as sa
from openpyxl.cell import Cell

In [13]:
# read db connection from .env file
from helpers.settings import get_settings

settings = get_settings()

In [14]:
engine = sa.create_engine(settings.postgresql_dsn.__str__())

In [15]:
df = pd.read_sql(
    """
SELECT i.individual_id, c.condition, ic.has_condition
FROM individual i
JOIN individual_condition ic
ON i.individual_id = ic.individual_id
JOIN condition c
ON ic.condition_id = c.condition_id
""",
    engine,
)
df.head()

,individual_id,condition,has_condition
0,1,Heart Structure Abnormality,False
1,1,Baseline/resting electrocardiogram abnormality,False
2,1,Polymorphic ventricular tachycardia,True
3,1,Exercise/stress induced polymorphic ventricula...,True
4,2,Heart Structure Abnormality,False


In [16]:
df2 = df.copy()
df2 = df2.pivot(index="individual_id", columns="condition",
                values="has_condition")
df2.reset_index(inplace=True)

df2.head()

condition,individual_id,Adult-onset primary generalised epilepsy,Arrhythmia at rest,Arrhythmogenic right ventricular cardiomyopathy,Arteriovenous malformation,Ascending aortic aneurysm,Atrial fibrillation,Atrial flutter,Atrial standstill,Atrial tachycardia,...,Sudden cardiac death,Sudden infant death syndrome,Supraventricular tachycardia,Syncope,Syncope exercise/stress induced,Third-degree atrioventricular block,Unspecified premature ventricular contractions,Ventricular fibrillation,Ventricular tachycardia (unspecified),Weight loss
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False,...,NaN,NaN,NaN,True,True,NaN,False,NaN,NaN,NaN
3,4,NaN,False,NaN,NaN,NaN,False,NaN,NaN,True,...,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# find the current working directory
import os
import IPython

notebook_path = IPython.get_ipython().getoutput('pwd')[0]

date = pd.Timestamp.now().strftime("%Y-%m-%d")

data_dir = os.path.join(notebook_path, '..', 'data', "database", date)

if not os.path.exists(data_dir):
    print(f'Creating directory: {data_dir}')
    os.makedirs(data_dir)

In [18]:
# save to csv
df_file = os.path.join(data_dir, "individual_conditions.csv")
df2.to_csv(df_file, index=False)

# Combine purified data with rest of condition data

In [19]:
df_all_condition = pd.read_sql(
    """
SELECT i.individual_id, c.condition, ic.age_of_onset, ic.description, ic.onset_symptoms, ic.age_of_presentation
FROM individual i
LEFT JOIN individual_condition ic
ON i.individual_id = ic.individual_id
LEFT JOIN condition c
ON ic.condition_id = c.condition_id
""",
    engine,
)
df_all_condition.head()

,individual_id,condition,age_of_onset,description,onset_symptoms,age_of_presentation
0,1,Heart Structure Abnormality,NaN,None,None,NaN
1,1,Baseline/resting electrocardiogram abnormality,NaN,None,None,NaN
2,1,Polymorphic ventricular tachycardia,NaN,None,None,NaN
3,1,Exercise/stress induced polymorphic ventricula...,NaN,None,None,NaN
4,2,Heart Structure Abnormality,NaN,None,None,NaN


In [20]:
df_all_condition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13687 entries, 0 to 13686
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   individual_id        13687 non-null  int64  
 1   condition            13687 non-null  object 
 2   age_of_onset         411 non-null    float64
 3   description          260 non-null    object 
 4   onset_symptoms       313 non-null    object 
 5   age_of_presentation  655 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 641.7+ KB


In [21]:
df_all_condition.groupby("individual_id")["description"].apply(
    lambda x: ", ".join(x.dropna())
).reset_index()

,individual_id,description
0,1,
1,2,
2,3,
3,4,
4,5,
...,...,...
1342,1350,2:1 noncompaction
1343,1352,
1344,1353,Abnormal RV relaxation
1345,1354,Prominent left ventricular trabeculation


In [22]:
df3_pivot = df_all_condition.pivot(
    index="individual_id",
    columns="condition",
    values=["age_of_onset", "onset_symptoms", "age_of_presentation"],
)
df3_pivot.reset_index(inplace=True)
# drop any columns that are all NaN
df3_pivot = df3_pivot.dropna(axis=1, how="all")
df3_pivot.columns = [
    "::".join(col).strip().strip(":") for col in df3_pivot.columns.values
]
# reset individual_id to a column
df3_pivot.set_index("individual_id", inplace=True)
# join the description column
df3_pivot = df3_pivot.join(
    df_all_condition.groupby("individual_id")["description"]
    .apply(lambda x: ", ".join(x.dropna()))
    .reset_index()
    .set_index("individual_id")
)
# rename the description column to "other conditions"
df3_pivot.rename(columns={"description": "other conditions"}, inplace=True)
df3_pivot.head()

,age_of_onset::Catecholaminergic polymorphic ventricular tachycardia 1,onset_symptoms::Catecholaminergic polymorphic ventricular tachycardia 1,age_of_presentation::Catecholaminergic polymorphic ventricular tachycardia 1,other conditions
individual_id,,,,
1,NaN,None,36.0,
2,NaN,none,41.0,
3,NaN,None,16.0,
4,2.0,None,NaN,
5,10.0,None,28.0,


In [23]:
# save to csv
df3_pivot.to_csv(
    os.path.join(data_dir, "individual_conditions_descriptions.csv"),
    index=False
)

## Family History Conditions

In [24]:
df4_pre = pd.read_sql(
    """
SELECT i.individual_id,
       c.condition,
       fmh.has_condition,
       kn.name AS relationship
FROM individual i
         LEFT JOIN family_history_record fhr
                   ON i.individual_id = fhr.individual_id
         LEFT JOIN condition c
                   ON fhr.condition_id = c.condition_id
         JOIN family_member_history fmh
                   ON fhr.family_history_record_id =
                      fmh.family_history_record_id
         JOIN kinship_name kn
                   ON fmh.kinship_name_id = kn.kinship_name_id
ORDER BY i.individual_id
""",
    engine,
)
df4_pre.head()

,individual_id,condition,has_condition,relationship
0,1,Sudden cardiac death,True,Mother
1,1,Sudden cardiac death,False,Father
2,10,Sudden cardiac death,False,Father
3,10,Sudden cardiac death,True,Mother
4,11,Sudden cardiac death,False,Father


In [25]:
df4_1_pivot = df4_pre.pivot(
    index="individual_id", columns=["condition", "relationship"],
    values="has_condition"
)
df4_1_pivot.reset_index(inplace=True)
df4_1_pivot.columns = [
    "::".join(col).strip().strip(":") for col in df4_1_pivot.columns.values
]
df4_1_pivot.set_index("individual_id", inplace=True)
df4_1_pivot.head()

,Sudden cardiac death::Mother,Sudden cardiac death::Father
individual_id,,
1,True,False
10,True,False
11,True,False
14,True,False
15,True,False


In [26]:
df4_2_pre = pd.read_sql(
    """
SELECT i.individual_id,
       c.condition,
       num_family_members
FROM individual i
         JOIN family_history_record fhr
                   ON i.individual_id = fhr.individual_id
         JOIN condition c
                   ON fhr.condition_id = c.condition_id
WHERE num_family_members IS NOT NULL
ORDER BY num_family_members DESC
""",
    engine,
)
df4_2_pre.head()

,individual_id,condition,num_family_members
0,881,Sudden cardiac death,36
1,877,Sudden cardiac death,36
2,875,Sudden cardiac death,36
3,873,Sudden cardiac death,36
4,871,Sudden cardiac death,36


In [27]:
df4_2_pivot = df4_2_pre.pivot(
    index="individual_id", columns="condition", values="num_family_members"
).reset_index()
df4_2_pivot.columns = [
    f"{col}::num_family_members" if col != "individual_id" else col
    for col in df4_2_pivot.columns.values
]
df4_2_pivot.set_index("individual_id", inplace=True)
df4_2_pivot.head()

,Sudden cardiac death::num_family_members
individual_id,
4,0
9,0
13,0
14,0
15,0


In [28]:
# combine the two dataframes - not all individuals will be in both
df4_pivot = df4_1_pivot.join(df4_2_pivot, how="outer")
df4_pivot.head()

,Sudden cardiac death::Mother,Sudden cardiac death::Father,Sudden cardiac death::num_family_members
individual_id,,,
1,True,False,NaN
4,NaN,NaN,0.0
9,NaN,NaN,0.0
10,True,False,NaN
11,True,False,NaN


In [29]:
# save to csv
df4_pivot.to_csv(
    os.path.join(data_dir, "individual_family_conditions.csv"),
)

## Treatments

In [30]:
df5 = pd.read_sql(
    """
SELECT i.individual_id,
       tr.treatment_taken,
       tr.effective,
       t.treatment_name
FROM individual i
         JOIN treatment_record tr
                   ON i.individual_id = tr.patient_id
         LEFT JOIN treatment t
                   ON tr.treatment_id = t.treatment_id
ORDER BY i.individual_id
""",
    engine,
)
df5.head()

,individual_id,treatment_taken,effective,treatment_name
0,2,True,False,Beta blocker
1,2,False,None,Verapamil
2,2,True,True,Flecainide
3,2,False,None,Enalapril
4,2,False,None,Left cardiac sympathetic denervation


In [31]:
df5_pivot = df5.pivot(
    index="individual_id",
    columns="treatment_name",
    values=["treatment_taken", "effective"],
)
df5_pivot.reset_index(inplace=True)
df5_pivot.columns = [
    "::".join(col).strip().strip(":") for col in df5_pivot.columns.values
]
df5_pivot.set_index("individual_id", inplace=True)
df5_pivot.head()

,treatment_taken::Beta blocker,treatment_taken::Catheter ablation,treatment_taken::Enalapril,treatment_taken::Flecainide,treatment_taken::Implantable cardioverter-defibrillator,treatment_taken::Left cardiac sympathetic denervation,treatment_taken::Verapamil,effective::Beta blocker,effective::Catheter ablation,effective::Enalapril,effective::Flecainide,effective::Implantable cardioverter-defibrillator,effective::Left cardiac sympathetic denervation,effective::Verapamil
individual_id,,,,,,,,,,,,,,
2,True,False,False,True,False,False,False,False,None,None,True,None,None,None
4,True,False,False,False,False,False,False,None,None,None,None,None,None,None
6,False,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
7,False,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
8,False,False,False,False,True,False,False,None,None,None,None,True,None,None


In [32]:
# save to csv
df5_pivot.to_csv(
    os.path.join(data_dir, "individual_treatments.csv"),
)

# Combine all data

In [55]:
df_rest = pd.read_sql(
    """
SELECT i.individual_id,
       iis.value              AS sex,
       iv.variant_id,
       z.zygosity,
       vi.variant_inheritance AS inheritance,
       v.hgvs_string,
       sv.p_hgvs_string,
       etp.name as protein_change,
       sv.sequence_variant_id,
       itp.publication_id,
       p.title,
       p.first_author,
       p.reference,
       p.doi,
       p.year,
       pd.name AS publication_database,
       pd.resource_uri,
       ptd.resource_id,
       i.extra_information
FROM individual i
         LEFT JOIN individual_sex iis
                   ON i.individual_sex_id = iis.individual_sex_id
         LEFT JOIN individual_variant iv
                   ON i.individual_id = iv.individual_id
         LEFT JOIN zygosity z
                   ON iv.zygosity_id = z.zygosity_id
         LEFT JOIN variant_inheritance vi
                   ON iv.variant_inheritance_id = vi.variant_inheritance_id
         LEFT JOIN variant v
                   ON iv.variant_id = v.variant_id
         LEFT JOIN individual_to_publication itp
                   ON i.individual_id = itp.individual_id
         LEFT JOIN publication p
                   ON itp.publication_id = p.publication_id
         LEFT JOIN sequence_variant sv
                   ON v.sequence_variant_id = sv.sequence_variant_id
         LEFT JOIN publication_to_database ptd
                   ON p.publication_id = ptd.publication_id
        LEFT JOIN publication_database pd
                   ON pd.database_id = ptd.database_id
        LEFT JOIN edit_type etp
                   ON sv.p_edit_type = etp.edit_type_id
ORDER BY i.individual_id;
""",
    engine,
)
df_rest["resource_uri"] = df_rest.apply(
    lambda x: f"{x['resource_uri']}{x['resource_id']}" if x[
        "resource_id"] else None,
    axis=1,
)
df_rest["doi_uri"] = df_rest.apply(
    lambda x: f"https://doi.org/{x['doi']}" if x["doi"] else None, axis=1
)
df_rest.head()

,individual_id,sex,variant_id,zygosity,inheritance,hgvs_string,p_hgvs_string,protein_change,sequence_variant_id,publication_id,title,first_author,reference,doi,year,publication_database,resource_uri,resource_id,extra_information,doi_uri
0,1,None,609,heterozygous,inherited,NM_001035.3:c.13564-41A>G,None,None,607.0,1,Familial Evaluation in Catecholaminergic Polym...,Van Der Werf C,None,10.1161/CIRCEP.112.970517,2012,PubMed,https://pubmed.ncbi.nlm.nih.gov/22787013,22787013,"{'basic_science': False, 'maternity_paternity_...",https://doi.org/10.1161/CIRCEP.112.970517
1,2,female,599,None,None,NM_001035.3:c.14885A>G,NP_001026.2:p.(Tyr4962Cys),Substitution,597.0,2,Flecainide therapy reduces exercise-induced ve...,"van der Werf, C",None,10.1016/j.jacc.2011.01.026,2011,PubMed,https://pubmed.ncbi.nlm.nih.gov/21616285,21616285,{'basic_science': False},https://doi.org/10.1016/j.jacc.2011.01.026
2,3,male,681,None,None,NM_001035.3:c.14173T>A,NP_001026.2:p.(Tyr4725Asn),Substitution,677.0,3,Structural abnormalities on cardiac magnetic r...,"Gerber, D",None,10.1016/j.jacep.2020.03.006,2020,PubMed,https://pubmed.ncbi.nlm.nih.gov/32553227,32553227,"{'basic_science': False, 'maternity_paternity_...",https://doi.org/10.1016/j.jacep.2020.03.006
3,4,female,3698,heterozygous,None,NM_001035.3:c.14174A>G,NP_001026.2:p.(Tyr4725Cys),Substitution,3671.0,4,Genetic Background of Catecholaminergic Polymo...,"Kawamura, M",None,10.1253/circj.cj-12-1460,2013,PubMed,https://pubmed.ncbi.nlm.nih.gov/23595086,23595086,{'basic_science': False},https://doi.org/10.1253/circj.cj-12-1460
4,5,male,3698,None,spontaneous,NM_001035.3:c.14174A>G,NP_001026.2:p.(Tyr4725Cys),Substitution,3671.0,5,Gender differences in the inheritance mode of ...,"Ohno, S.",None,10.1371/journal.pone.0131517,2015,PubMed,https://pubmed.ncbi.nlm.nih.gov/26114861,26114861,"{'basic_science': False, 'maternity_paternity_...",https://doi.org/10.1371/journal.pone.0131517


In [36]:
df_all = (
    df_rest.set_index("individual_id")
    .join(df2)
    .join(df3_pivot)
    .join(df4_pivot)
    .join(df5_pivot)
)
df_all.drop(columns="individual_id", inplace=True)
# set the index name to individual_id
df_all.index.name = "individual_id"
df_all.head()

,sex,variant_id,zygosity,inheritance,hgvs_string,p_hgvs_string,publication_id,title,first_author,reference,...,treatment_taken::Implantable cardioverter-defibrillator,treatment_taken::Left cardiac sympathetic denervation,treatment_taken::Verapamil,effective::Beta blocker,effective::Catheter ablation,effective::Enalapril,effective::Flecainide,effective::Implantable cardioverter-defibrillator,effective::Left cardiac sympathetic denervation,effective::Verapamil
individual_id,,,,,,,,,,,,,,,,,,,,,
1,None,609,heterozygous,inherited,NM_001035.3:c.13564-41A>G,None,1,Familial Evaluation in Catecholaminergic Polym...,Van Der Werf C,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,female,599,None,None,NM_001035.3:c.14885A>G,NP_001026.2:p.(Tyr4962Cys),2,Flecainide therapy reduces exercise-induced ve...,"van der Werf, C",None,...,False,False,False,False,None,None,True,None,None,None
3,male,681,None,None,NM_001035.3:c.14173T>A,NP_001026.2:p.(Tyr4725Asn),3,Structural abnormalities on cardiac magnetic r...,"Gerber, D",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,female,3698,heterozygous,None,NM_001035.3:c.14174A>G,NP_001026.2:p.(Tyr4725Cys),4,Genetic Background of Catecholaminergic Polymo...,"Kawamura, M",None,...,False,False,False,None,None,None,None,None,None,None
5,male,3698,None,spontaneous,NM_001035.3:c.14174A>G,NP_001026.2:p.(Tyr4725Cys),5,Gender differences in the inheritance mode of ...,"Ohno, S.",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
import datetime

date = datetime.datetime.now().strftime("%Y-%m-%d")

date

'2024-07-18'

## Join with individual_mv table's exons and structure domains

In [40]:
indiv_mv = pd.read_sql(
    """
SELECT i.individual_id, i.exon_start, i.exon_end, i.structure_domains, i.p_edit_type
FROM individuals_mv i
""",
    engine,
    dtype={
        "exon_start": "Int64",
        "exon_end": "Int64",
    },
)
indiv_mv.head()

,individual_id,exon_start,exon_end,structure_domains,p_edit_type
0,1,<NA>,<NA>,None,None
1,2,105,106,"[{'structure_id': 19, 'structure_domain': 'Act...",Substitution
2,3,99,100,"[{'structure_id': 19, 'structure_domain': 'Act...",Substitution
3,4,99,100,"[{'structure_id': 25, 'structure_domain': 'Pse...",Substitution
4,5,99,100,"[{'structure_id': 25, 'structure_domain': 'Pse...",Substitution


In [41]:
# domains
domains_df = pd.read_csv("./data_commit/ryr2_subdomains.csv")
domains_df

,domains,subdomains,subdomain_precedence
0,NTD,NTD-A,NaN
1,SPRY,NTD-B,NaN
2,JSol,NSol,NaN
3,BSol,SPRY1,NaN
4,SCLP,SPRY2,NaN
5,CSol,SPRY3,NaN
6,TaF,RY1&2,NaN
7,TM,JSol,NaN
8,CTD,BSol1,NaN
9,NaN,BSol2,NaN


In [46]:
def find_domain(structure_domain: list[dict], domains: set[str]):
    if not structure_domain:
        return None

    in_domains = []
    for d in structure_domain:
        # example value: [{'structure_id': 27, 'structure_domain': 'C-terminal domain ', 'structure_domain_symbol': 'CTD'}, {'structure_id': 19, 'structure_domain': 'Activation core and channel', 'structure_domain_symbol': None}]
        domain_symbol = d["structure_domain_symbol"]

        if domain_symbol is None or pd.isna(domain_symbol):
            continue

        if domain_symbol in domains:
            in_domains.append(domain_symbol)

    if len(in_domains) == 0:
        print(
            f"No domains found: {structure_domain}. Setting to full name if structure_domain is length 1."
        )
        return return_first_full_domain_name(structure_domain)

    if len(in_domains) == 1:
        return in_domains[0]

    raise ValueError(f"Multiple domains found: {in_domains}")


def find_subdomain(
        structure_domain: list[dict], subdomain_with_precedence: dict[str, bool]
):
    if not structure_domain:
        return None

    in_subdomains = []
    for d in structure_domain:

        domain_symbol = d["structure_domain_symbol"]

        if domain_symbol is None or pd.isna(domain_symbol):
            continue

        if domain_symbol in subdomain_with_precedence:
            in_subdomains.append(domain_symbol)

    if len(in_subdomains) == 0:
        print(
            f"No subdomains found: {structure_domain}. Setting to full name if structure_domain is length 1."
        )

        return return_first_full_domain_name(structure_domain)

    if len(in_subdomains) == 1:
        return in_subdomains[0]

    # make sure only 1 has precedence
    with_precedence = [d for d in in_subdomains if subdomain_with_precedence[d]]

    if len(with_precedence) == 1:
        return with_precedence[0]

    raise ValueError(f"Multiple subdomains found: {in_subdomains}")


def return_first_full_domain_name(structure_domain: list[dict]):
    if not structure_domain:
        return None

    if len(structure_domain) == 1:
        return structure_domain[0]["structure_domain"]

    print("WARNING: Multiple domains found.")
    return None


df_all2 = df_all[
    [
        "age_of_onset::Catecholaminergic polymorphic ventricular tachycardia 1",
        "hgvs_string",
        "p_hgvs_string",
    ]
].join(indiv_mv.set_index("individual_id"))

df_all2["domain"] = df_all2["structure_domains"].apply(
    lambda x: find_domain(
        x, set(d.strip() for d in domains_df["domains"].values if pd.notna(d))
    )
)

df_all2["subdomain"] = df_all2["structure_domains"].apply(
    lambda x: find_subdomain(
        x,
        {
            d.strip(): p if pd.notna(p) else False
            for d, p in zip(
            domains_df["subdomains"].values,
            domains_df["subdomain_precedence"].values,
        )
            if pd.notna(d)
        },
    )
)


def determine_exon(variant: str, exon_start: int | None, exon_end: int | None):
    if pd.isna(exon_start) or pd.isna(exon_end):
        return None

    if exon_end - exon_start == 1:
        return exon_start

    print(
        f"WARNING: VARIANT {variant} SPANS MULTIPLE EXONS {exon_start} - {exon_end - 1}"
    )

    return f"{exon_start}-{exon_end - 1}"


df_all2["exon"] = df_all2.apply(
    lambda x: determine_exon(x["hgvs_string"], x["exon_start"], x["exon_end"]),
    axis=1
)

df_all2.rename(
    columns={
        "age_of_onset::Catecholaminergic polymorphic ventricular tachycardia 1": "age_of_onset::CPVT1"
    },
    inplace=True,
)

df_all2.drop(columns=["structure_domains", "exon_start", "exon_end"],
             inplace=True)

df_all2.head()

No domains found: [{'structure_id': 19, 'structure_domain': 'Activation core and channel', 'structure_domain_symbol': None}]. Setting to full name if structure_domain is length 1.
No domains found: [{'structure_id': 19, 'structure_domain': 'Activation core and channel', 'structure_domain_symbol': None}]. Setting to full name if structure_domain is length 1.
No domains found: [{'structure_id': 19, 'structure_domain': 'Activation core and channel', 'structure_domain_symbol': None}]. Setting to full name if structure_domain is length 1.
No subdomains found: [{'structure_id': 19, 'structure_domain': 'Activation core and channel', 'structure_domain_symbol': None}]. Setting to full name if structure_domain is length 1.
No subdomains found: [{'structure_id': 19, 'structure_domain': 'Activation core and channel', 'structure_domain_symbol': None}]. Setting to full name if structure_domain is length 1.
No subdomains found: [{'structure_id': 19, 'structure_domain': 'Activation core and channel', 

,age_of_onset::CPVT1,hgvs_string,p_hgvs_string,p_edit_type,domain,subdomain,exon
individual_id,,,,,,,
1,NaN,NM_001035.3:c.13564-41A>G,None,None,None,None,None
2,NaN,NM_001035.3:c.14885A>G,NP_001026.2:p.(Tyr4962Cys),Substitution,CTD,CTD,105
3,NaN,NM_001035.3:c.14173T>A,NP_001026.2:p.(Tyr4725Asn),Substitution,TM,pVSD,99
4,2.0,NM_001035.3:c.14174A>G,NP_001026.2:p.(Tyr4725Cys),Substitution,TM,pVSD,99
5,10.0,NM_001035.3:c.14174A>G,NP_001026.2:p.(Tyr4725Cys),Substitution,TM,pVSD,99


In [48]:
df_all_4real = df_all.join(
    df_all2[
        [
            "domain",
            "subdomain",
            "exon",
        ]
    ]
)
df_all_4real.head()

,sex,variant_id,zygosity,inheritance,hgvs_string,p_hgvs_string,publication_id,title,first_author,reference,...,effective::Beta blocker,effective::Catheter ablation,effective::Enalapril,effective::Flecainide,effective::Implantable cardioverter-defibrillator,effective::Left cardiac sympathetic denervation,effective::Verapamil,domain,subdomain,exon
individual_id,,,,,,,,,,,,,,,,,,,,,
1,None,609,heterozygous,inherited,NM_001035.3:c.13564-41A>G,None,1,Familial Evaluation in Catecholaminergic Polym...,Van Der Werf C,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
2,female,599,None,None,NM_001035.3:c.14885A>G,NP_001026.2:p.(Tyr4962Cys),2,Flecainide therapy reduces exercise-induced ve...,"van der Werf, C",None,...,False,None,None,True,None,None,None,CTD,CTD,105
3,male,681,None,None,NM_001035.3:c.14173T>A,NP_001026.2:p.(Tyr4725Asn),3,Structural abnormalities on cardiac magnetic r...,"Gerber, D",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM,pVSD,99
4,female,3698,heterozygous,None,NM_001035.3:c.14174A>G,NP_001026.2:p.(Tyr4725Cys),4,Genetic Background of Catecholaminergic Polymo...,"Kawamura, M",None,...,None,None,None,None,None,None,None,TM,pVSD,99
5,male,3698,None,spontaneous,NM_001035.3:c.14174A>G,NP_001026.2:p.(Tyr4725Cys),5,Gender differences in the inheritance mode of ...,"Ohno, S.",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TM,pVSD,99


In [43]:
from contextlib import contextmanager
import openpyxl
import openpyxl.utils


@contextmanager
def open_workbook(filename: str):
    wb = None

    try:
        wb = openpyxl.load_workbook(filename)
        yield wb
        wb.save(filename)
    except Exception as e:
        print("Something went wrong: ", e)
    finally:
        if wb:
            wb.close()

In [50]:
from openpyxl.styles import Font

excel_filename = os.path.join(data_dir, f"individual_all_data-{date}.xlsx")

# save to csv
df_all_4real.to_excel(os.path.join(data_dir, excel_filename), sheet_name="all_data")

with open_workbook(excel_filename) as wb:
    sheet = wb["all_data"]

    col_to_idx = {
        row[i].value: i
        for row in sheet.iter_rows(min_row=1, max_row=1)
        for i in range(len(row))
    }
    for i, row in enumerate(sheet.iter_rows(min_row=2)):
        for col in ["resource_uri", "doi_uri"]:
            cell: Cell = row[col_to_idx[col]]

            if isinstance(cell.value, str) and cell.value.startswith("http"):
                cell.hyperlink = cell.value
                cell.font = Font(color="0000FF", underline="single")